In this notebook we use [SX.jl](https://github.com/JuliaReach/SX.jl) to convert SpaceEx files into a `HybridSystem` whose modes are `MathematicalSystems` types.

In [1]:
using Revise, SX, MathematicalSystems

┌ Info: Recompiling stale cache file /Users/forets/.julia/compiled/v1.1/SX/qIGUA.ji for SX [2fbac1b3-cb3a-5de3-9d8f-ec145dc5cab0]
└ @ Base loading.jl:1184


### Lorenz model

In [2]:
file = "hyst/lorenz/lorenz.xml"
readsxmodel(file, raw_dict=true)

ErrorException: read 2 components, but models with more than one component are not yet implemented; try flattening the model

A current limitation of SX is that it can only handle SpaceEx models with one network component.

The model has two components, one called `main` and other called `sys`. `main` contains the dynamics, and `sys` is just a binding between the keys and variable names `x, y, z`.

In this case we can either flatten the model (instructions in the [SX docs](https://juliareach.github.io/SX.jl/latest/examples/examples.html#Remarks-1)) or remove `sys` manually. Let's just remove the `sys` component and try again.

In [3]:
file = "hyst/lorenz/lorenz_modif.xml"
M = readsxmodel(file, raw_dict=true)

Dict{String,Any} with 10 entries:
  "automaton"        => LightAutomaton{SimpleDiGraph{Int64},SimpleEdge{Int64}}(…
  "invariants"       => Array{Expr,1}[[:(())]]
  "transitionlabels" => Set(String[])
  "variables"        => OrderedCollections.OrderedDict(:x=>Dict("local"=>false,…
  "guards"           => Array{Expr,1}[]
  "flows"            => Array{Expr,1}[[:(x' = 10 + (y - x)), :(y' = x * (28 - z…
  "assignments"      => Array{Expr,1}[]
  "switchings"       => HybridSystems.AbstractSwitching[]
  "nlocations"       => 1
  "ntransitions"     => 0

In [4]:
f = M["flows"]

1-element Array{Array{Expr,1},1}:
 [:(x' = 10 + (y - x)), :(y' = x * (28 - z) - y), :(z' = x * y - 2.6667z)]

Possible paths:

- convert into a `BlackBoxContinuousSystem` with a `@taylorize` of a `MultivariatePolynomial`
- convert into a `PolynomialContinuousSystem`

In [5]:
using MultivariatePolynomials, DynamicPolynomials

In [6]:
@polyvar x y z

(x, y, z)

In [7]:
e = f[1][1].args[2]

:(10 + (y - x))

In [8]:
dx = Vector{Polynomial}(undef, 3)
dx[1] = (10 + (y - x))
dx[2] = x * (28 - z) - y
dx[3] = x * y - 2.6667z;

### Van der Pol

In [9]:
file = "hyst/vanderpol/vanderpol_althoff_modif.xml"
Hvdp = readsxmodel(file, raw_dict=true)

Dict{String,Any} with 10 entries:
  "automaton"        => LightAutomaton{SimpleDiGraph{Int64},SimpleEdge{Int64}}(…
  "invariants"       => Array{Expr,1}[[:(())]]
  "transitionlabels" => Set(String[])
  "variables"        => OrderedCollections.OrderedDict(:x=>Dict("local"=>false,…
  "guards"           => Array{Expr,1}[]
  "flows"            => Array{Expr,1}[[:(x' = y), :(y' = (1 - x * x) * y - x)]]
  "assignments"      => Array{Expr,1}[]
  "switchings"       => HybridSystems.AbstractSwitching[]
  "nlocations"       => 1
  "ntransitions"     => 0

In [10]:
f = Hvdp["flows"]

1-element Array{Array{Expr,1},1}:
 [:(x' = y), :(y' = (1 - x * x) * y - x)]

In [11]:
Hvdp["variables"]

OrderedCollections.OrderedDict{Symbol,Dict{String,Any}} with 2 entries:
  :x => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"=>"any","i…
  :y => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"=>"any","i…

In [12]:
Hvdp["invariants"]

1-element Array{Array{Expr,1},1}:
 [:(())]

### Van der Pol hybridized

In the file `out_vanderpol_hybridized_plot_modif.xml`:

- delete the `vanderpol_althoff_net` component
- delete the note

In [13]:
file = "hyst/vanderpol/out_vanderpol_hybridized_plot_modif.xml"
H = readsxmodel(file, raw_dict=true);

┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 3 → 4 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 12 → 15 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 15 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 30 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 30 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 32 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 32 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 32 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 42 → 45 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 60 → 61 is ignored
└ @ SX /Users/fo

...

┌ Warning: field labelposition in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 354 → 355 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 351 → 354 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 351 → 354 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 351 → 356 is ignore

┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 375 → 378 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 375 → 378 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 375 → 380 is ignore

┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 393 → 394 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 393 → 394 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 393 → 394 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 393 → 394 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 393 → 394 is ignore

┌ Warning: field labelposition in transition 402 → 405 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 405 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignore

┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 417 → 420 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 417 → 420 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 417 → 422 is ignore

In [14]:
length(H["flows"])

422

In [15]:
H["variables"]

OrderedCollections.OrderedDict{Symbol,Dict{String,Any}} with 5 entries:
  :x           => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :y           => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :_tt         => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :_x_interval => Dict{String,Any}("local"=>false,"controlled"=>false,"dynamics…
  :_y_interval => Dict{String,Any}("local"=>false,"controlled"=>false,"dynamics…

In [16]:
H["flows"][4]

3-element Array{Expr,1}:
 :(x' = 0 + _x_interval)
 :(y' = 0 + _y_interval)
 :(_tt' = 0)            

In [17]:
H["invariants"][1]

8-element Array{Expr,1}:
 :(2.1556820658251534x + -4.072711973886765y <= -5.58130540874347)
 :(x >= 1.2)                                                      
 :(x <= 1.6478249031507648)                                       
 :(y >= 2.052442405578342)                                        
 :(y <= 2.3699999999999997)                                       
 :(_x_interval = 0)                                               
 :(_y_interval <= 0.24163929002508056)                            
 :(_y_interval >= 0)                                              

In [21]:
 readsxmodel(file, raw=true);

┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 14 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 14 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 16 is ignored
└ @ SX /Users/forets/.

┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/fo

...

┌ Warning: field labelposition in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 291 → 297 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 291 → 297 is ignore

┌ Warning: field labelposition in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 303 → 304 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 299 → 303 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 299 → 303 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 299 → 305 is ignore

┌ Warning: field labelposition in transition 303 → 310 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 303 → 310 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 312 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 312 is ignore

┌ Warning: field labelposition in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 311 → 317 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 311 → 317 is ignore

┌ Warning: field labelposition in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 323 → 324 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 323 → 324 is ignore

In [18]:
H_CLCCS = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem);

┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 3 → 4 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 15 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 15 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 12 → 17 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 12 → 17 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 30 → 31 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 30 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 30 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 32 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 39 → 44 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 39 → 44 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 45 → 46 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 45 → 46 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 54 → 59 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 60 → 61 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 60 → 61 is ignored
└ @ SX /Users/fo

...

┌ Warning: field labelposition in transition 372 → 377 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 372 → 377 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 378 → 379 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 378 → 379 is ignore

┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 387 → 392 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 393 → 394 is ignore

┌ Warning: field labelposition in transition 405 → 406 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 405 → 406 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 405 → 406 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 405 → 406 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 405 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 405 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 402 → 407 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 402 → 407 is ignore

┌ Warning: field labelposition in transition 414 → 419 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 414 → 419 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 420 → 421 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 420 → 421 is ignore

ErrorException: guard x <= 1.2 in transition 1 contains a combination of state variables and input variables, and conversion to a system of type ConstrainedLinearControlContinuousSystem is not possible

In [19]:
file = "hyst/tank/out_tank_hybridized_plot_modif.xml"
H = readsxmodel(file, ST=ConstrainedLinearControlContinuousSystem);

┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 1 → 2 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 8 is ignored
└ @ SX /Users/forets/.julia/dev/SX

┌ Warning: field labelposition in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 13 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 7 → 14 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 7 → 14 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 16 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 16 is ignored
└ @ SX /Users/forets/.

┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 15 → 21 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 15 → 21 is ignored
└ @ SX /Users/fo

┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 27 → 28 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 27 → 28 is ignored
└ @ SX /Users/fo

...

┌ Warning: field labelposition in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 308 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 308 is ignore

┌ Warning: field labelposition in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 307 → 313 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 307 → 313 is ignore

┌ Warning: field labelposition in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 320 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 315 → 319 is ignore

Excessive output truncated after 524516 bytes.┌ Warning: field labelposition in transition 319 → 325 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 325 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 319 → 326 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 319 → 326 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 327 → 328 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 327 → 328 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field labelposition in transition 327 → 328 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field middlepoint in transition 327 → 328 is ignored
└ @ SX /Users/forets/.julia/dev/SX/src/parse.jl:350
┌ Warning: field l

ErrorException: guard x1 <= 1.7 in transition 1 contains a combination of state variables and input variables, and conversion to a system of type ConstrainedLinearControlContinuousSystem is not possible

In [20]:
H["variables"]

OrderedCollections.OrderedDict{Symbol,Dict{String,Any}} with 5 entries:
  :x           => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :y           => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :_tt         => Dict{String,Any}("local"=>false,"controlled"=>true,"dynamics"…
  :_x_interval => Dict{String,Any}("local"=>false,"controlled"=>false,"dynamics…
  :_y_interval => Dict{String,Any}("local"=>false,"controlled"=>false,"dynamics…